In [1]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
from scraper import *
import numpy as np
import scipy.sparse as sp
%load_ext autoreload
%autoreload 2

# display all pandas columns
pd.set_option('display.max_columns', 100)

# Friendship Matrix

Scraping of initiative tables

In [4]:
df_business = pd.read_csv('data/Business.csv')
df_role = pd.read_csv('data/BusinessRole.csv')
df_status = pd.read_csv('data/BusinessStatus.csv')
df_type = pd.read_csv('data/BusinessType.csv')
df_member = pd.read_csv('data/MemberCouncil.csv')

/Users/Jonas/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Explanation

* **Business** is the table of initiatives
* **BusinessRole** table of all the person who act for or against a business. IMPORTANT! **BusinessRole.MemberCouncilNumber** link to **MemberCouncil.PersonNumber**. **BusinessShortNumber** link businesses and roles.
    * Auteur
    * Cosignataire
    * Porte-parole
    * Opposant(e)
    * Repreneur
    
* **BusinessType** contain all 18 types of businesses. As the type is reapeated in **Business.BusinessType** and **Business.BusinessTypeName**, it is not useful for us.
* **BusinessStatus** give the current status of a business. Again, this information is given in **Business.BusinessStatusText**, so not useful


In [5]:
print(df_business.shape)
df_business.head(2)

(38802, 44)


,Unnamed: 0,BusinessShortNumber,BusinessStatus,BusinessStatusDate,BusinessStatusText,BusinessType,BusinessTypeAbbreviation,BusinessTypeName,Category,Description,DocumentationText,DraftText,FederalCouncilProposal,FederalCouncilProposalDate,FederalCouncilProposalText,FederalCouncilResponseText,FirstCouncil1,FirstCouncil1Abbreviation,FirstCouncil1Name,FirstCouncil2,FirstCouncil2Abbreviation,FirstCouncil2Name,ID,InitialSituation,IsLeadingDepartment,Language,Modified,MotionText,Proceedings,ReasonText,ResponsibleDepartment,ResponsibleDepartmentAbbreviation,ResponsibleDepartmentName,SubmissionCouncil,SubmissionCouncilAbbreviation,SubmissionCouncilName,SubmissionDate,SubmissionLegislativePeriod,SubmissionSession,SubmittedBy,SubmittedText,TagNames,Tags,Title
0,0,11.758,27,2000-09-26T00:00:00,Liquidé,3,Iv.ct.,Initiative cantonale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19110758,NaN,True,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,4.0,DFI,Département de l'intérieur,NaN,NaN,NaN,1973-08-15T00:00:00,39.0,4311,Berne,<text><p>La législation fédérale doit être dév...,NaN,NaN,Médicaments. Législation
1,1,78.222,27,1981-03-10T00:00:00,Liquidé,4,Iv.pa.,Initiative parlementaire,NaN,Rapport de la Commission du 27.08.1979,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,NaN,NaN,NaN,19780222,NaN,NaN,FR,2010-12-26T13:31:48.703,NaN,NaN,NaN,NaN,NaN,NaN,1.0,CN,Conseil national,1978-06-05T00:00:00,40.0,4311,NaN,NaN,NaN,NaN,Code pénal. Interruption de la grossesse (Gira...


In [6]:
print(df_role.shape)
df_role.head()

(356969, 19)


,Unnamed: 0,BusinessNumber,BusinessShortNumber,BusinessSubmissionDate,BusinessTitle,BusinessType,BusinessTypeAbbreviation,BusinessTypeName,CantonNumber,CommitteeNumber,ID,IdExternal,Language,MemberCouncilNumber,Modified,ParlGroupNumber,ReturnType,Role,RoleName
0,0,19110758,11.758,1973-08-15T00:00:00,Médicaments. Législation,3,Iv.ct.,Initiative cantonale,2.0,NaN,8cf23579-9fd7-4be3-a338-b654da878d8c,NaN,FR,NaN,2010-12-26T13:07:38.53,NaN,1,7,Auteur
1,1,19850227,85.227,1985-02-07T00:00:00,Droit des assurances sociales,4,Iv.pa.,Initiative parlementaire,NaN,NaN,f2ccb9d9-b2fd-4fda-abeb-43e61f020921,NaN,FR,148.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur
2,2,19880208,88.208,1988-11-25T00:00:00,Abandon du projet de centrale nucléaire de Graben,3,Iv.ct.,Initiative cantonale,11.0,NaN,4c34e2a2-891e-4b28-8917-170c4c3d2313,NaN,FR,NaN,2010-12-26T13:07:38.53,NaN,1,7,Auteur
3,3,19890245,89.245,1989-11-27T00:00:00,Service civil,4,Iv.pa.,Initiative parlementaire,NaN,NaN,4156c078-6a3c-4279-95f7-4802374379a6,NaN,FR,114.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur
4,4,19890255,89.255,1989-12-15T00:00:00,Réserves d'or de la Banque nationale,4,Iv.pa.,Initiative parlementaire,NaN,NaN,081932bd-7632-4ef4-8b51-a66ee174be98,NaN,FR,102.0,2010-12-26T13:07:38.53,NaN,0,7,Auteur


## Get number of initiative the guy are author

In [7]:
member_number_initiatives  = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Auteur')].groupby('MemberCouncilNumber').size()

member_number_initiatives = member_number_initiatives.to_frame(name='Times_author').head()

In [8]:
member_number_initiatives.head()

,Times_author
MemberCouncilNumber,
1.0,42
2.0,6
6.0,1
7.0,10
8.0,5


In [10]:
member_number_initiatives.shape

(5, 1)

## SciPy sparse matrix

Create a Scipy relation with attribute:
* row: autor
* column: co-signer
* data: number he co-sign for him

We can also do it with a pandas, and limite the number of person for juste the one still in function. Anyway...

In [12]:
# create Autor and Co-signer table

Cosignataire = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Cosignataire')]
Cosignataire = Cosignataire[['BusinessNumber', 'MemberCouncilNumber']]
Cosignataire = Cosignataire.astype(int)
print("Cosignataires: ", Cosignataire.shape)

Auteur = df_role.loc[(df_role.MemberCouncilNumber.notnull()) & (df_role.RoleName == 'Auteur')]
Auteur = Auteur[['BusinessNumber', 'MemberCouncilNumber']]
Auteur = Auteur.astype(int)
print("Auteurs: ", Auteur.shape)
Auteur.head(2)

Cosignataires:  (316548, 2)
Auteurs:  (31095, 2)


,BusinessNumber,MemberCouncilNumber
1,19850227,148
3,19890245,114


In [43]:
# Fill the sparse matrix

max_id = df_member.PersonNumber.max()
friends = sp.lil_matrix((max_id, max_id), dtype=np.int32)

def add_to_friend(autor, coautors):
    for coautor in coautors:
        friends[autor, coautor] += 1
        
        
def fill_matrix(Auteur, Cosignataire):
    for (auteur_num, bus_num) in zip(Auteur.MemberCouncilNumber, Auteur.BusinessNumber):
        coautors = Cosignataire.loc[Cosignataire.BusinessNumber == bus_num]['MemberCouncilNumber']
        if coautors.size != 0:
            add_to_friend(auteur_num, coautors)
        
fill_matrix(Auteur, Cosignataire)
friends.nonzero()

(array([   1,    1,    1, ..., 4209, 4209, 4209], dtype=int32),
 array([   1,    8,    9, ..., 4200, 4202, 4208], dtype=int32))

In [44]:
# return a pandas.DataFrame with the friend of the person, ordered by times they cosign something

def get_friends(friend_matrix, person_num):
    x = friend_matrix[person_num,:].nonzero()[1]
    y = friend_matrix[person_num,:].data[0]

    df = pd.DataFrame({'Person_num':x, 'times_cosigner':y })
    df = df.sort_values(by='times_cosigner', ascending=False)
    df = df.reset_index(drop=True)
    return df
   
get_friends(friends, 2).head()

,Person_num,times_cosigner
0,14,2
1,226,2
2,217,2
3,135,2
4,108,2


In [16]:
Cosignataire

,BusinessNumber,MemberCouncilNumber
20,19913398,130
25,19913398,133
26,19913398,129
27,19913398,143
30,19913398,140
31,19913398,1
33,19910432,210
34,19913398,224
37,19913398,102
42,19913398,69


# Compiling the _Cosignataire_ info into an adjacency matrix for d3

In [84]:
adj = pd.read_csv('data/adj.csv')

In [85]:
adj.rename(columns={'PersonNumber': 'PersonIdCode'}, inplace=True)

In [86]:
adj.set_index('PersonIdCode', inplace=True)

In [87]:
adj.head(2)

,2287,2292,2310,2143,2147,2323,2328,2370,2372,2385,2405,2417,2061,2501,2129,2510,2525,2530,2534,2536,2541,2546,2561,2023,2565,2569,2573,2575,2580,2588,2600,2621,2623,2618,2589,2627,2632,2594,2593,2597,2630,2583,2584,2617,2585,2614,2616,2638,2608,2629,...,3067,3069,3074,3076,3078,3080,3081,3082,3085,3089,3090,3091,3092,3095,3096,3097,3098,3099,3101,3102,3056,3075,3060,3083,3058,3072,3086,3059,3061,3065,3070,3073,3079,3088,3100,3071,3077,3084,3087,3094,3066,3068,3103,3104,3105,3106,3107,3108,3109,3110
PersonIdCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2287,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2292,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
people = pd.read_csv('data/people_jonas.csv').set_index('PersonIdCode')

In [36]:
people.loc[2023].PersonNumber

724

In [118]:
def fill_adj(adj, people):
    # getting a list of active members (we're only interested in them)
    active = people.PersonNumber.tolist()
    # going through an empty adj matrix with PersonIdCodes as rows and columns
    for row in adj.iterrows():
        person_id = row[0]
        # converting from PersonIdCode to PersonNumber for friends search
        person_number = people.loc[person_id].PersonNumber
        # searching co-sign friends w/ the function defined above
        friends_matrix = get_friends(friends, person_number)
        # looping through friends to fill the matrix
        for friend in friends_matrix.iterrows():
            # checking if active
            if friend[1].Person_num in active:
                # converting from PersonNumber to PersonIdCode
                friend_id = people.loc[people.PersonNumber == friend[1].Person_num].index.tolist()[0]
                # Updating matrix
                adj.loc[person_id, str(friend_id)] = friend[1].times_cosigner

In [89]:
fill_adj(adj, people)

## WARNING: the adjacency matrix is **NOT** symmetric

For now, I'll be only considering rows in the viz, but might make sense to consider columns, dunno.

In [116]:
adj.head(2)

,2287,2292,2310,2143,2147,2323,2328,2370,2372,2385,2405,2417,2061,2501,2129,2510,2525,2530,2534,2536,2541,2546,2561,2023,2565,2569,2573,2575,2580,2588,2600,2621,2623,2618,2589,2627,2632,2594,2593,2597,2630,2583,2584,2617,2585,2614,2616,2638,2608,2629,...,3067,3069,3074,3076,3078,3080,3081,3082,3085,3089,3090,3091,3092,3095,3096,3097,3098,3099,3101,3102,3056,3075,3060,3083,3058,3072,3086,3059,3061,3065,3070,3073,3079,3088,3100,3071,3077,3084,3087,3094,3066,3068,3103,3104,3105,3106,3107,3108,3109,3110
PersonIdCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2287,0,0,0,3,0,3,1,1,2,0,1,1,1,3,1,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2292,7,1,27,0,29,69,1,0,58,23,0,1,2,4,2,8,1,7,0,52,46,2,0,0,1,1,5,0,10,24,5,16,11,46,0,51,0,1,0,3,2,2,54,29,4,3,15,55,0,0,...,2,3,9,11,10,10,4,3,1,4,6,9,9,11,7,10,8,6,6,9,1,2,0,0,0,0,0,0,0,4,0,0,1,0,0,0,2,0,0,0,2,0,1,1,0,0,0,0,2,1


In [93]:
adj.to_csv('data/adj_cosign.csv')

In [117]:
adj.to_json('data/adj_cosign.json', orient='index')